In [1]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CacheConfig
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.parameters import ParameterString, ParameterFloat, ParameterInteger
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
import boto3

C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\pydantic\_internal\_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/30/25 14:26:20] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=590193;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=425316;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\tochi\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
# Set up session and role
region = "us-east-2"  
sagemaker_session = sagemaker.Session()

account = boto3.client('sts').get_caller_identity().get('Account')
role = f"arn:aws:iam::{account}:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole"  

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=74513;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=248764;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=402226;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=636385;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

In [3]:
# Define pipeline parameters
years_to_filter = ParameterString(name="Historical_Years", default_value="10")
cache_config = CacheConfig(enable_caching=True, expire_after="1d") #Cache configuration

In [4]:
# S3 bucket for storing intermediate data
inference_bucket = "s3://aws-portfolio-projects/snp500-data/inference_data/"

## Data Ingestion Step 

In [5]:
# Use an SKLearnProcessor for data ingestion

data_ingestion_processor = SKLearnProcessor(
    framework_version="1.0-1",
    command=["python3"],
    role=role,
    instance_count=1,
    instance_type="ml.m5.large"
)

# Data ingestion step
ingestion_step = ProcessingStep(
    name="DataIngestion",
    processor=data_ingestion_processor,
    inputs=[],
    outputs=[
        sagemaker.processing.ProcessingOutput(
                                              output_name="ingested",
                                              source="/opt/ml/processing/output",
                                              destination= inference_bucket),
    ],
    code="inference_scripts/data_ingestion.py",  # This script fetches data from the yahoo finance API
    cache_config=cache_config,
    job_arguments=[
        "--years-to-filter", years_to_filter
    ]
)

[01/30/25 14:26:21] INFO     Defaulting to only available Python version: py3                     ]8;id=425665;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=686637;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\image_uris.py#608\608]8;;\

## Data Processing Step

In [6]:
data_preprocessor = SKLearnProcessor(
    framework_version="1.0-1", 
    role=role,
    instance_type="ml.m5.large",
    instance_count=1
)

# Data processing step
processing_step = ProcessingStep(
    name="DataPreprocessing",
    processor=data_preprocessor,
    inputs=[
        sagemaker.processing.ProcessingInput(
            source=ingestion_step.properties.ProcessingOutputConfig.Outputs["ingested"].S3Output.S3Uri,
            destination="/opt/ml/processing/input"
        )
    ],
    outputs=[
        sagemaker.processing.ProcessingOutput(output_name="processed", 
                                              source="/opt/ml/processing/output/train", 
                                              destination=inference_bucket)

    ],
    code="inference_scripts/data_processing.py", 
    cache_config=cache_config,
    job_arguments=[
        "--input_dir", "/opt/ml/processing/input/",
        "--output_dir", "/opt/ml/processing/output/train"
    ]
   
)

[01/30/25 14:26:22] INFO     Defaulting to only available Python version: py3                     ]8;id=371417;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=390472;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\image_uris.py#608\608]8;;\

## Inference Step

In [23]:
# Create a Lambda function for prediction
lambda_role = "arn:aws:iam::930627915954:role/sagemaker-pipeline-lambda-role"
lambda_client = boto3.client("lambda", region_name=region)
lambda_function_name = "sagemaker-stock-prediction-lambda" 
# Lambda function for invoking the SageMaker endpoint
lambda_step = LambdaStep(
    name="InvokeSageMakerEndpoint",
    lambda_func=Lambda(
        function_name=lambda_function_name,
        execution_role_arn =lambda_role,
        script="inference_scripts/lambda_helper.py",
        handler="lambda_helper.lambda_handler" 
    ),
    inputs={
        "processed_data_s3_uri": "s3://aws-portfolio-projects/snp500-data/inference_data/sp500_processed.csv",
        "endpoint_name": "stock-market-prediction-endpoint",  
        "dynamodb_table_name": "stock-prediction-data"
    },
)

In [24]:
# Define the pipeline
pipeline = Pipeline(
    name="StockInferencePipeline",
    parameters=[years_to_filter],
    steps=[ingestion_step, processing_step,lambda_step],
    sagemaker_session=sagemaker_session,
)

# Execute the pipeline
pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.wait()

[01/30/25 16:24:12] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=621032;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=432355;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

[01/30/25 16:24:13] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=279645;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=734841;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

[01/30/25 16:24:14] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=325377;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=303147;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=900062;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=334384;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 # Execute the pipeline                                                                      │
│   10 pipeline.upsert(role_arn=role)                                                              │
│   11 execution = pipeline.start()                                                                │
│ ❱ 12 execution.wait()                                                                            │
│   13                                                                                             │
│                                                                                                  │
│ C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\pipeline.py:934 │
│ in wait                                                                                          │
│                                                                                                  │
│    931 │   │   waiter = botocore.waiter.create_waiter_with_client(                               │
│    932 │   │   │   waiter_id, model, self.sagemaker_session.sagemaker_client                     │
│    933 │   │   )                                                                                 │
│ ❱  934 │   │   waiter.wait(PipelineExecutionArn=self.arn)                                        │
│    935 │                                                                                         │
│    936 │   def result(self, step_name: str):                                                     │
│    937 │   │   """Retrieves the output of the provided step if it is a ``@step`` decorated func  │
│                                                                                                  │
│ C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\waiter.py:55 in wait      │
│                                                                                                  │
│    52 │   # Waiter.wait method. This is needed to attach a docstring to the                      │
│    53 │   # method.                                                                              │
│    54 │   def wait(self, **kwargs):                                                              │
│ ❱  55 │   │   Waiter.wait(self, **kwargs)                                                        │
│    56 │                                                                                          │
│    57 │   wait.__doc__ = WaiterDocstring(                                                        │
│    58 │   │   waiter_name=waiter_name,                                                           │
│                                                                                                  │
│ C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\waiter.py:374 in wait     │
│                                                                                                  │
│   371 │   │   │   │   return                                                                     │
│   372 │   │   │   if current_state == 'failure':                                                 │
│   373 │   │   │   │   reason = f'Waiter encountered a terminal failure state: {acceptor.explan   │
│ ❱ 374 │   │   │   │   raise WaiterError(                                                         │
│   375 │   │   │   │   │   name=self.name,                                                        │
│   376 │   │   │   │   │   reason=reason,                                                         │
│   377 │   │   │   │   │   last_response=response,                                                │
╰────────────────────────────────────────────────────────────

## Schedule Pipeline

In [ ]:
import pytz
from datetime import datetime, timedelta
from sagemaker.workflow.triggers import PipelineSchedule

# Set US Central Time (UTC-6)
central_tz = pytz.timezone('US/Central')

# Set the schedule time to 11 PM US Central Time
schedule_time = central_tz.localize(datetime.now().replace(hour=23, minute=0, second=0, microsecond=0))

# Convert to UTC
schedule_time_utc = schedule_time.astimezone(pytz.utc)

# Define the pipeline schedule to run every day at 11 PM US Central Time (which is 5 AM UTC)
pipeline_schedule = PipelineSchedule(
    name='inference-pipeline-daily-schedule',
    start_date=schedule_time_utc,
    at=schedule_time_utc,
    enabled=True
)

### What to do tomorrow:

1. write the pipeline so that it reads input from previous pipeline
2. redeploy an endpoint
3. run the pipeline and troubleshoot if needed.
4. When the pipeline works, delete dynamodb all the necessary stuff on sagemaker. 